In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [25]:
# As a training data set, download eng-jpn parallel corpus from tatoeba.org
!wget http://downloads.tatoeba.org/exports/sentences.tar.bz2 -P ../data/
!tar -jxf ../data/sentences.tar.bz2 -C ../data/

!wget http://downloads.tatoeba.org/exports/links.tar.bz2 -P ../data/
!tar -jxf ../data/links.tar.bz2 -C ../data/

In [27]:
sentences = {}
sentence_lang = {}
with open("../data/sentences.csv", encoding="utf-8") as f:
    for line in f:
        i, lang, sentence = line.strip("\n").split("\t")
        sentence_lang[i] = lang
        if lang == "eng" or lang == "jpn":
            sentences[i] = sentence

In [28]:
translations = []
with open("../data/links.csv") as f:
    for line in f:
        i, j = line.strip("\n").split("\t")
        if i in sentences and j in sentences and sentence_lang[i] == "eng" and sentence_lang[j] == "jpn":
            translations.append((sentences[i], sentences[j]))

In [29]:
## Sample translations
translations[:10]

[("Let's try something.", '何かしてみましょう。'),
 ('I have to go to sleep.', '私は眠らなければなりません。'),
 ('I have to go to sleep.', 'そろそろ寝なくちゃ。'),
 ("Today is June 18th and it is Muiriel's birthday!",
  '今日は６月１８日で、ムーリエルの誕生日です！'),
 ('Muiriel is 20 now.', 'ムーリエルは２０歳になりました。'),
 ('Muiriel is 20 now.', '今、ムーリイエルは２０歳だ。'),
 ('The password is "Muiriel".', 'パスワードは「Muiriel」です。'),
 ('I will be back soon.', 'すぐに戻ります。'),
 ('I will be back soon.', 'まもなく私は戻って来ます。'),
 ('I will be back soon.', 'すぐ戻ります。')]

In [30]:
eng_vocabs = set()
jpn_vocabs = set()
with open("../translation.tsv", "w", encoding="utf-8") as f:
    for eng, jpn in translations:
        f.write("%s\t%s\n" % (eng, jpn))
        eng_vocabs |= set(eng)
        jpn_vocabs |= set(jpn)

In [31]:
eng_dict = {}
with open("../eng_vocabs.txt", "w", encoding="utf-8") as f:
    for i, w in enumerate(eng_vocabs):
        f.write("%s\n" % w)
        eng_dict[w] = str(i + 4)

jpn_dict = {}
with open("../jpn_vocabs.txt", "w", encoding="utf-8") as f:
    for i, w in enumerate(jpn_vocabs):
        f.write("%s\n" % w)
        jpn_dict[w] = str(i + 4)

In [32]:
PAD = "0"
GO = "1"
UNK = "2"
EOS = "3"

In [35]:
max_len = 200

with open("../data/translation_id.tsv_tmp", "w", encoding="utf-8") as f:
    for eng, jpn in translations:
        if len(eng) <= max_len and len(jpn) <= max_len:
            eng = " ".join([eng_dict[w] for w in eng] + [EOS])
            jpn = " ".join([GO] + [jpn_dict[w] for w in jpn] + [EOS])
            f.write("%s\t%s\n" % (eng, jpn))

In [36]:
#Shuffle training data

!shuf ../data/translation_id.tsv_tmp > ../data/translation_id.tsv
!rm ../data/translation_id.tsv_tmp